In [1]:
import pandas as pd
import numpy as np
from lets_plot import *
LetsPlot.setup_html()

In [2]:
df = pd.read_csv('battle_results.csv')

df['p1win'] = df['result'] == 1
df['p2win'] = df['result'] == 2
df['tie'] = df['result'] == 0

df['p1name'] = df['p1name'].str.title()
df['p2name'] = df['p2name'].str.title()

df = df.drop(columns=['Unnamed: 0'])

df.head(5)

,player1,player2,result,p1name,p2name,p1win,p2win,tie
0,0,0,2,Scyther,Scyther,False,True,False
1,0,1,2,Scyther,Pinsir,False,True,False
2,0,2,2,Scyther,Moltres,False,True,False
3,0,3,2,Scyther,Moltres Ex,False,True,False
4,0,4,2,Scyther,Heatmor,False,True,False


In [3]:
pokemon_df = pd.read_csv('../simulation/data/pokemon-tcg-with-fossils.csv')
pokemon_df.head(5)

,id,stage,name,from,hp,type,ability,a1cost,a1name,a1damage,a1bonus,a1bonusDmg,a2cost,a2name,a2damage,a2bonus,a2bonusDmg,weakness,retreat,rarity
0,ga1,0,bulbasaur,NaN,70,grass,NaN,gx,vine whip,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fire,1,1d
1,ga2,1,ivysaur,bulbasaur,90,grass,NaN,gxx,razor leaf,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fire,2,2d
2,ga3,2,venusaur,ivysaur,160,grass,NaN,ggxx,mega drain,80.0,heal,30.0,NaN,NaN,NaN,NaN,NaN,fire,3,3d
3,ga4,2,venusaur EX,ivysaur,190,grass,NaN,gxx,razor leaf,60.0,NaN,NaN,ggxx,giant bloom,100.0,heal,30.0,fire,3,4d
4,ga5,0,caterpie,NaN,50,grass,NaN,x,find a friend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fire,1,1d


In [4]:
evo_names = df.groupby('player1')['p1name'].first().reset_index().rename(columns={'player1': 'id', 'p1name': 'name'})

evo_names.head(5)

,id,name
0,0,Scyther
1,1,Pinsir
2,2,Moltres
3,3,Moltres Ex
4,4,Heatmor


In [5]:
max_battles = int((df['player1'].max() + 1) * 2) - 1
max_battles

615

In [42]:
player_wins = df.groupby('player1')[['p1win', 'tie']].sum().rename(columns={'p1win': 'player_win', 'tie': "player_tie"})
opponent_wins = df[df['player1'] != df['player2']].groupby('player2')[['p2win', 'tie']].sum().rename(columns={'p2win': 'opponent_win', 'tie': "opponent_tie"})

wins = player_wins.merge(opponent_wins, left_index=True, right_index=True).reset_index().rename(columns={'player1': 'id'})
wins['wins'] = wins['player_win'] + wins['opponent_win']
wins['ties'] = wins['player_tie'] + wins['opponent_tie']
wins['losses'] = max_battles - (wins['wins'] + wins['ties'])

wins = wins.set_index('id').merge(evo_names.set_index('id'), right_index=True, left_index=True).reset_index()

wins['win_perc'] = (wins['wins'] / max_battles * 100).round(2)

wins['is_ex'] = wins['name'].str.contains(' Ex')

wins['pk_index'] = wins['name'].apply(lambda name: pokemon_df[pokemon_df['name'].str.lower() == name.lower()].first_valid_index())
wins['type'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'type'].title())
wins['stage'] = wins['pk_index'].apply(lambda id: {0: 'Basic', 1: 'Stage 1', 2: 'Stage 2'}[pokemon_df.loc[id, 'stage']])
wins['ability'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'ability'].title() if pokemon_df.loc[id, 'ability'] is not np.nan else 'None')
wins['has_ability'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'ability'] is not np.nan)
wins['hp'] = wins['pk_index'].apply(lambda id: pokemon_df.loc[id, 'hp'])

wins.head(5)

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
0,0,44,0,87,0,131,0,484,Scyther,21.30,False,24,Grass,Basic,None,False,70
1,1,136,0,218,0,354,0,261,Pinsir,57.56,False,25,Grass,Basic,None,False,90
2,2,147,0,227,0,374,0,241,Moltres,60.81,False,45,Fire,Basic,None,False,100
3,3,206,0,255,0,461,0,154,Moltres Ex,74.96,True,46,Fire,Basic,None,False,140
4,4,59,0,101,0,160,0,455,Heatmor,26.02,False,47,Fire,Basic,None,False,80


In [27]:
def get_losses(id:int) -> set[int]:
    p1 = df[(df['player1'] == id) & ~(df['p1win'])]
    p2 = df[(df['player2'] == id) & ~(df['p2win'])]
    return set([int(item) for item in set(p1['player2'].unique()) | set(p2['player1'].unique())])

In [28]:
loss74 = get_losses(74)
loss75 = get_losses(75)

eggLosses = loss74 & loss75

named_evo_list = lambda s: [evo_names.loc[item, 'name'].title() for item in s]

print(f"Common losses:           {named_evo_list(eggLosses)}")
print(f"Egg 74 unique losses:    {named_evo_list(loss74 - eggLosses)}")
print(f"Egg 75 unique losses:    {named_evo_list(loss75 - eggLosses)}")

Common losses:           ['Exeggutor Ex', 'Exeggutor Ex', 'Rampardos', 'Bastidon', 'Aerodactyl Ex', 'Rapidash', 'Infernape Ex']
Egg 74 unique losses:    ['Glaceon', 'Mismagius Ex']
Egg 75 unique losses:    ['Seaking', 'Exeggutor', 'Rapidash', 'Rapidash', 'Kricketune', 'Celebi Ex', 'Salazzle']


In [29]:
(df['result'].value_counts() / df['result'].count() * 100).round(2)

result
2    55.48
1    44.42
0     0.10
Name: count, dtype: float64

In [30]:
wins[['id', 'wins', 'win_perc', 'name']].sort_values('wins', axis=0, ascending=False).head(20)

,id,wins,win_perc,name
74,74,607,98.70,Exeggutor Ex
75,75,601,97.72,Exeggutor Ex
289,289,590,95.93,Magnezone
300,300,590,95.93,Rampardos
176,176,585,95.12,Aerodactyl Ex
290,290,585,95.12,Magnezone
286,286,577,93.82,Infernape Ex
99,99,567,92.20,Starmie Ex
73,73,564,91.71,Exeggutor
243,243,559,90.89,Blastoise Ex


In [32]:
def losses_df(df, id):
    return df[((df['player1'] == id) & ~(df['p1win'])) | ((df['player2'] == id) & ~(df['p2win']))]

In [33]:
losses_df(df, 300)

,player1,player2,result,p1name,p2name,p1win,p2win,tie
22476,72,300,1,Exeggutor,Rampardos,True,False,False
22784,73,300,1,Exeggutor,Rampardos,True,False,False
23092,74,300,1,Exeggutor Ex,Rampardos,True,False,False
23400,75,300,1,Exeggutor Ex,Rampardos,True,False,False
23708,76,300,1,Whimsicott,Rampardos,True,False,False
53584,173,300,1,Primape,Rampardos,True,False,False
53892,174,300,1,Primape,Rampardos,True,False,False
54200,175,300,1,Primape,Rampardos,True,False,False
54508,176,300,1,Aerodactyl Ex,Rampardos,True,False,False
60360,195,300,1,Floatzel,Rampardos,True,False,False


In [34]:
wins[['id', 'player_win', 'name']].sort_values('player_win', axis=0, ascending=False).head(20)

,id,player_win,name
74,74,307,Exeggutor Ex
300,300,306,Rampardos
75,75,305,Exeggutor Ex
289,289,302,Magnezone
290,290,300,Magnezone
176,176,299,Aerodactyl Ex
286,286,287,Infernape Ex
73,73,283,Exeggutor
99,99,283,Starmie Ex
261,261,283,Kabutops


In [35]:
df[df['tie']].groupby('p1name')['result'].count().reset_index().sort_values('result', ascending=False)

,p1name,result
8,Lumineon,18
6,Hitmonlee,12
1,Chatot,10
9,Manaphy,10
10,Mespirit,10
3,Ditto,8
11,Mew Ex,8
5,Fan Rotom,6
0,Aerodactyl,5
2,Cinccino,3


In [36]:
wins[wins['wins'] <= 5][['id', 'name', 'wins']].sort_values('wins', ascending=False)

,id,name,wins
165,165,Lumineon,3
19,19,Hitmonlee,2
166,166,Lumineon,1
39,39,Chatot,0
48,48,Manaphy,0
52,52,Mespirit,0


In [21]:
(
    ggplot(
        df,
        aes(
            x='result'
        )
    )
    + geom_bar()
    + scale_x_discrete(breaks=[2, 1, 0], labels=['Player 2', 'Player 1', 'Tie'])
    + labs(
        title='Total Battle Results',
        x='Winner',
        y='Count'
    )
)

In [37]:
(
    ggplot(
        (wins.groupby('is_ex')['wins'].sum() / wins.groupby('is_ex')['wins'].count()).round().reset_index(),
        aes(
            x='is_ex',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + scale_x_discrete(breaks=[False, True], labels=['Non-EX', 'EX'])
    + labs(
        title='Average Wins by EX',
        x='EX?',
        y='Avg. Wins',
    )
)

In [38]:
(
    ggplot(
        (wins.groupby('type')['wins'].sum() / wins.groupby('type')['wins'].count()).round().reset_index().sort_values('wins', ascending=True),
        aes(
            x='type',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + labs(
        title='Wins by Type',
        x='Type',
        y='Avg. Wins'
    )
)

In [33]:
(
    ggplot(
        (wins.groupby('stage')['wins'].sum() / wins.groupby('stage')['wins'].count()).round().reset_index().sort_values('wins'),
        aes(
            x='stage',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + labs(
        title='Wins by Final Evolution Stage',
        x='Stage',
        y='Avg. Wins'
    )
)

In [43]:
(
    ggplot(
        (wins.groupby('has_ability')['wins'].sum() / wins.groupby('has_ability')['wins'].count()).round().reset_index().sort_values('wins'),
        aes(
            x='has_ability',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + scale_x_discrete(breaks=[False, True], labels=['No', 'Yes'])
    + labs(
        title='Wins by Having an Ability',
        x='Has Ability?',
        y='Avg. Wins'
    )
)

In [40]:
(
    ggplot(
        wins,
        aes(
            x='hp',
            y='wins'
        )
    )
    + geom_boxplot()
    + geom_point(
        data=wins[wins['ties'] > 0],
        color='red'
    )
    + labs(
        title='Wins by Final Evolution HP',
        x='HP',
        y='Wins'
    )
)

In [48]:
wins[(wins['ties'] > 0) & (wins['hp'] == 80) & (wins['wins'] > 202)]

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
66,66,92,6,118,5,210,11,394,Fan Rotom,34.15,False,485,Normal,Basic,None,False,80


In [51]:
wins[(wins['ties'] > 0) & (wins['hp'] == 130) & (wins['wins'] > 357)]

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
33,33,187,8,254,7,441,15,159,Mew Ex,71.71,True,298,Psychic,Basic,None,False,130


In [50]:
wins[(wins['ties'] > 0) & (wins['hp'] == 140) & (wins['wins'] > 427)]

,id,player_win,player_tie,opponent_win,opponent_tie,wins,ties,losses,name,win_perc,is_ex,pk_index,type,stage,ability,has_ability,hp
261,261,283,2,258,1,541,3,71,Kabutops,87.97,False,158,Fighting,Stage 2,None,False,140


In [ ]:
(
    ggplot(
        (wins.groupby('ability')['wins'].sum() / wins.groupby('ability')['wins'].count()).round().reset_index().sort_values('wins'),
        aes(
            x='ability',
            y='wins'
        )
    )
    + geom_bar(stat='identity')
    + labs(
        title='Wins by Ability',
        x='Ability',
        y='Avg. Wins'
    )
)

In [52]:
df[(df['player1'] == df['player2']) & (df['result'] == 1)]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
309,1,1,1,Pinsir,Pinsir,True,False,False
5253,17,17,1,Mewtwo Ex,Mewtwo Ex,True,False,False
9888,32,32,1,Dedenne,Dedenne,True,False,False
11433,37,37,1,Druddigon,Druddigon,True,False,False
12360,40,40,1,Carnivine,Carnivine,True,False,False
...,...,...,...,...,...,...,...,...
93009,301,301,1,Gallade Ex,Gallade Ex,True,False,False
93318,302,302,1,Gallade Ex,Gallade Ex,True,False,False
93627,303,303,1,Bastidon,Bastidon,True,False,False
93936,304,304,1,Garchomp,Garchomp,True,False,False


In [57]:
df[(df['player1'] == df['player2']) & (df['result'] == 0)]

,player1,player2,result,p1name,p2name,p1win,p2win,tie
5871,19,19,0,Hitmonlee,Hitmonlee,False,False,True
8034,26,26,0,Ditto,Ditto,False,False,True
12051,39,39,0,Chatot,Chatot,False,False,True
14832,48,48,0,Manaphy,Manaphy,False,False,True
16068,52,52,0,Mespirit,Mespirit,False,False,True
20394,66,66,0,Fan Rotom,Fan Rotom,False,False,True
50985,165,165,0,Lumineon,Lumineon,False,False,True
51294,166,166,0,Lumineon,Lumineon,False,False,True
80649,261,261,0,Kabutops,Kabutops,False,False,True


In [56]:
int(df[(df['player1'] == df['player2'])]['player1'].count())

308